In [1]:
from time import time
from glob import glob
import tifffile as tiff
import pandas as pd
import numpy as np
import cv2

import sys
sys.path.append('..')
import util

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

log_board = util.diagnostics.LogBoard('log_dir', 6005)
log_board.launch()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_dir = '/root/data'
model_pth = './bin/models/edge_model_trained.pt'
scan_folders = [
    f'{data_dir}/train/kidney_1_dense/',
]

/tmp/ipykernel_264727/1740922428.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

E0130 06:51:36.746982 140547308561024 program.py:298] TensorBoard could not bind to port 6005, it was already in use
ERROR: TensorBoard could not bind to port 6005, it was already in use


In [4]:
patch_size = 8,256,256
dataset = util.data.SenNet(
    patch_size,
    guarantee_vessel=0.5,
    samples=[
        "/train/kidney_2",
        "/train/kidney_3_dense"
    ]
)

Loading /train/kidney_2/images from cache
Loading /train/kidney_3_dense/images from cache
Loading /train/kidney_2/labels from cache
Loading /train/kidney_3_dense/labels from cache


In [5]:
dataset.scans[0].shape, dataset.scans[1].shape

(torch.Size([1, 2217, 1041, 1511]), torch.Size([1, 501, 1706, 1510]))

In [4]:
patch_size = 8,256,256
dataset = util.data.SenNet(
    patch_size,
    guarantee_vessel=0.5,   
    samples=[
        "/train/kidney_1_dense",
        # "/train/kidney_3_sparse"
    ]
)
dloader = DataLoader(dataset, batch_size=4, shuffle=True)

Loading /train/kidney_1_dense/images from cache


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

E0127 20:09:37.900740 140092181800128 program.py:298] TensorBoard could not bind to port 6005, it was already in use
ERROR: TensorBoard could not bind to port 6005, it was already in use


Loading /train/kidney_1_dense/labels from cache


In [5]:
model = util.UNet3P(
    in_f=8,
    layers=[16, 32, 32, 32, 64, 64, 64],
    block_depth=4,
    connect_depth=6,
    conv=util.nn.Conv2DNormed,
    pool_fn=nn.MaxPool2d,
    resize_kernel=(2,2),
    upsample_mode='bilinear',
    norm_fn=nn.BatchNorm2d,
    dropout=(nn.Dropout2d, 0.1)
).to(device)
model.load_state_dict(torch.load(model_pth, map_location=device))
model.requires_grad_(False)
model.eval()

UNet3P(
  (input_norm): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (down_blocks): ModuleList(
    (0): Sequential(
      (0): ConvBlock(
        (layers): ModuleList(
          (0): Sequential(
            (0): Dropout2d(p=0.1, inplace=False)
            (1): Conv2DNormed(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): GELU(approximate='none')
          )
          (1-3): 3 x Sequential(
            (0): Dropout2d(p=0.1, inplace=False)
            (1): Conv2DNormed(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): GELU(approximate='none')
          )
        )
      )
    )
    (1): Sequential(
      (0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (1): ConvBl

In [89]:
def fill_segmentation_outlines(tensor: np.ndarray, max_opening: int) -> np.ndarray:
    """
    Fill in the outlines of shapes in a 2D binary tensor.

    Args:
    tensor (np.ndarray): Input tensor of shape [B, F, W, H] containing binary segmentation outlines.
    max_opening (int): Maximum size of opening in the shapes to be filled.

    Returns:
    np.ndarray: Tensor with filled outlines.
    """
    B, F, W, H = tensor.shape
    filled_tensor = np.zeros_like(tensor)
    
    for b in range(B):
        for f in range(F):
            # Apply morphological closing to fill gaps up to 'max_opening' size
            kernel = np.ones((max_opening, max_opening), np.uint8)
            closing = cv2.morphologyEx(tensor[b, f], cv2.MORPH_CLOSE, kernel)

            # Fill the shapes
            for w in range(W):
                for h in range(H):
                    if closing[w, h] == 1:
                        cv2.floodFill(closing, None, (h, w), 1)
            
            filled_tensor[b, f] = closing

    return filled_tensor

def fill_segmentation(binary_image: np.ndarray) -> np.ndarray:
    """
    Fills in the shapes in a binary segmentation image.
    
    Parameters:
    - binary_image: A binary image (numpy array) where the shapes to fill are in white (255) on a black (0) background.
    
    Returns:
    - A numpy array with the same dimensions as binary_image with the shapes filled in.
    """
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create an output image that is all black
    filled_image = np.zeros_like(binary_image)
    
    # Fill the found contours in the output image
    cv2.drawContours(filled_image, contours, -1, (255), thickness=cv2.FILLED)
    
    return filled_image

def fill_outline(outline: torch.Tensor) -> torch.Tensor:
    outline = (outline[0, 0] * 255).numpy().astype(np.uint8)

    contours, _ = cv2.findContours(outline, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_image = np.zeros_like(outline)
    cv2.drawContours(filled_image, contours, -1, (255), thickness=cv2.FILLED)
    return torch.Tensor(filled_image).bool().unsqueeze(0).unsqueeze(0)

In [93]:
x, y, _ = next(iter(dloader))

x, y = x.float().to(device), y.float().to(device)
x, y = x.view(x.shape[0],x.shape[1]*x.shape[2],x.shape[3],x.shape[4]), y[:,:,y.shape[2]//2,:,:]

# Compute output
p_y = [torch.sigmoid(logits) for logits in model(x)]
pred_masks = [(p > 0.5) for p in p_y]

pred = pred_masks[-1][0].unsqueeze(0).cpu()
target = y[0].unsqueeze(0).cpu()

# filled = (pred[0, 0] * 255).to(torch.uint8).numpy()
filled = fill_outline(pred)
# filled = torch.from_numpy(filled).unsqueeze(0).unsqueeze(0)

util.Display.view([
    util.Display(pred),
    util.Display(filled),
    util.Display(target)
])

interactive(children=(IntSlider(value=0, description='i', max=0), Output()), _dom_classes=('widget-interact',)…

In [95]:
pred.shape

torch.Size([1, 1, 256, 256])

In [92]:
filled.max

<function Tensor.max>

In [34]:
filled.dtype

dtype('float32')

In [54]:
filled.min()

0.0

interactive(children=(IntSlider(value=0, description='i', max=0), Output()), _dom_classes=('widget-interact',)…